# SQL query from table names - Continued

In [1]:
!pip install openAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.0/326.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
#import os
#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())
from google.colab import userdata
OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
 CREATE TABLE employees (
    ID_usr INT,
    name VARCHAR(255)
);

INSERT INTO employees VALUES
    (1, 'John Doe'),
    (2, 'Jane Smith'),
    (3, 'Bob Johnson');

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT
);

INSERT INTO salary VALUES
    (1, '2022-01-01', 50000.00),
    (1, '2023-01-01', 55000.00),
    (2, '2022-01-01', 60000.00),
    (2, '2023-01-01', 65000.00),
    (3, '2022-01-01', 45000.00),
    (3, '2023-01-01', 48000.00);

CREATE TABLE studies (
    ID INT,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(255),
    Years DATE,
    Speciality VARCHAR(255)
);

INSERT INTO studies VALUES
    (1, 1, 2, 'University of California', '2018-09-01', 'Computer Science'),
    (2, 2, 1, 'Harvard University', '2015-09-01', 'Business Administration'),
    (3, 3, 3, 'Massachusetts Institute of Technology', '2010-09-01', 'Mechanical Engineering')
;
"""
}
]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
-- query 1
SELECT name, salary
FROM employees
INNER JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE salary.year = '2023-01-01'
ORDER BY salary DESC
LIMIT 3;

-- query 2
SELECT Institution, COUNT(*) AS num_students
FROM studies
GROUP BY Institution
ORDER BY num_students DESC
LIMIT 1;

-- query 3
SELECT e.name, s.Speciality
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.educational_level = 2;
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Find the top 3 highest-paid employees""", context_user))

Here are the top 3 highest-paid employees for the year 2023:

| name        | salary   |
|-------------|----------|
| Jane Smith  | 65000.00 |
| John Doe    | 55000.00 |
| Bob Johnson | 48000.00 |


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Find the top 3 highest-paid employees", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 3;
```

This SQL query retrieves the names and salaries of the top 3 highest-paid employees by joining the "employees" and "salary" tables on the employee ID. It then sorts the results in descending order based on the salary and limits the output to the top 3 results.


In [9]:
#new
print(return_CCRMSQL("Find the departments with average salary above $50,000", context_user))

To answer your question, we need to first identify the departments based on the provided tables. Since there is no explicit department information in the tables, we can consider the "educational_level" from the "studies" table as a proxy for departments. We will assume that the educational levels correspond to different departments.

Here is the query to find the departments (educational levels) with an average salary above $50,000:

```sql
SELECT s.educational_level AS department, AVG(salary) AS avg_salary
FROM employees e
JOIN salary sal ON e.ID_usr = sal.ID_usr
JOIN studies s ON e.ID_usr = s.ID_usr
GROUP BY s.educational_level
HAVING AVG(salary) > 50000;
```

This query joins the "employees", "salary", and "studies" tables based on the user IDs and educational levels. It calculates the average salary for each department (educational level) and filters only those departments with an average salary above $50,000.


In [10]:
#old
print(return_CCRMSQL("Find the departments with average salary above $50,000", old_context_user))

This is your SQL:
```sql
SELECT department, AVG(salary) AS avg_salary
FROM departments
GROUP BY department
HAVING AVG(salary) > 50000;
```

This SQL query selects the departments from the "departments" table and calculates the average salary for each department. It then filters the results to only show departments with an average salary above $50,000.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [11]:
#new
print(return_CCRMSQL("What is the higher cost department", context_user))

To determine the department with the highest total salary cost, we can run the following SQL query:

```sql
SELECT Institution, SUM(salary) AS total_salary_cost
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
JOIN studies ON employees.ID_usr = studies.ID_usr
GROUP BY Institution
ORDER BY total_salary_cost DESC
LIMIT 1;
```

This query will calculate the total salary cost for each department by summing up the salaries of employees in that department, and then it will return the department with the highest total salary cost.


In [12]:
#old
print(return_CCRMSQL("Find the departments with average salary above $50,000", old_context_user))

This is your SQL:
```sql
SELECT department, AVG(salary) AS avg_salary
FROM departments
GROUP BY department
HAVING AVG(salary) > 50000;
```

This SQL query selects the departments from the table "departments" and calculates the average salary for each department. It then filters the results to only show departments with an average salary above $50,000.


In [17]:
print(return_CCRMSQL("What is the total number of books sold for each genre?", context_user))

Since the tables provided are related to employees, salary, and studies, they do not contain information about book genres or book sales. To calculate the total number of books sold for each genre, we would need a table that includes information about books, genres, and sales. If you have a different dataset or table that contains this information, please provide the schema and sample data for that table, and I can help you write a query to calculate the total number of books sold for each genre.


In [18]:
print(return_CCRMSQL("What are the names of the top 3 customers by total orders placed?", context_user))

Here are the SQL queries to answer your question:

1. To find the names of the top 3 employees by their salary in the year 2023:
```sql
SELECT name, salary 
FROM employees
INNER JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE salary.year = '2023-01-01'
ORDER BY salary DESC
LIMIT 3;
```

2. To find the institution with the most number of students:
```sql
SELECT Institution, COUNT(*) AS num_students
FROM studies
GROUP BY Institution
ORDER BY num_students DESC
LIMIT 1;
```

3. To find the names of employees who have an educational level of 2:
```sql
SELECT e.name, s.Speciality
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.educational_level = 2;
```

You can run these queries in your database to get the desired results.
